In [1]:
import requests
import json
import os
from dotenv import load_dotenv
import pandas as pd
import datetime
import time
import csv
import psycopg2
import sqlalchemy
from sqlalchemy import create_engine
import schedule
from google.cloud.sql.connector import Connector
import pg8000.native

In [2]:
connector = Connector()

# function to return the database connection
def getconn() -> pg8000.native.Connection:
    conn: pg8000.connections.Connection = connector.connect(
        os.environ.get("google_cloud_project"),
        "pg8000",
        user=os.environ.get("google_cloud_user"),
        password=os.environ.get("google_cloud_pass"),
        db="bicimad_worker"
    )
    return conn

# create connection pool
pool = sqlalchemy.create_engine(
    "postgresql+pg8000://",
    creator=getconn,
)

In [4]:
def get_token():
    load_dotenv('./.env')
    email = os.environ.get("email")
    password = os.environ.get("password")
    client_id = os.environ.get("X-ClientId")
    pass_key = os.environ.get("passKey")
    url = "https://openapi.emtmadrid.es/v3/mobilitylabs/user/login/"
    headers = {"email": email, "password" : password}
    response = requests.get(url, headers=headers)
    return response.content

In [26]:
def update_estaciones(): #Esta función hace UPDATE de los datos que no cambian de las estaciones en la base de datos
    connector = Connector()

    # function to return the database connection
    def getconn() -> pg8000.native.Connection:
        conn: pg8000.connections.Connection = connector.connect(
            os.environ.get("google_cloud_project"),
            "pg8000",
            user=os.environ.get("google_cloud_user"),
            password=os.environ.get("google_cloud_pass"),
            db="bicimad_worker"
        )
        return conn

    # create connection pool
    pool = sqlalchemy.create_engine(
        "postgresql+pg8000://",
        creator=getconn,
    )

    load_dotenv('./.env')
    token = os.environ.get("access_token")
    url = "https://openapi.emtmadrid.es/v3/transport/bicimad/stations/"
    headers = {"accessToken" : token}
    stations = requests.get(url, headers = headers).json()
    date_and_time = datetime.datetime.now()
    date_and_time_formated = date_and_time.strftime("%Y-%m-%d %H:%M:%S")
    date_and_time_formated2 = date_and_time.strftime("%Y%m%d%H%M%S")
    estaciones = pd.DataFrame(stations["data"])
    estaciones["last_updated"] = date_and_time_formated
    estaciones[["longitude", "latitude"]] = estaciones["geometry"].apply(lambda x: pd.Series(x["coordinates"]))
    estaciones = estaciones.drop(["activate", "virtualDelete", "tipo_estacionPBSC", "geofence", "activate", "geometry", "integrator", "virtual_bikes", "virtual_bikes_num", "geofenced_capacity", "bikesGo"], axis=1)
    estaciones['coordinates'] = list(zip(estaciones['longitude'], estaciones['latitude']))
    update_estaciones = estaciones[["address", "code_district", "code_suburb", "id", "number", "total_bases", "last_updated", "longitude", "latitude", "coordinates"]]
    update_estaciones.to_csv(f'../data_csv/estaciones_{date_and_time_formated2}.csv')
    with pool.connect() as conn:
        update_estaciones.to_sql('estaciones', con=conn, if_exists='replace', index=False)
    return update_estaciones

In [21]:
connector = Connector()

# function to return the database connection
def getconn() -> pg8000.native.Connection:
    conn: pg8000.connections.Connection = connector.connect(
        os.environ.get("google_cloud_project"),
        "pg8000",
        user=os.environ.get("google_cloud_user"),
        password=os.environ.get("google_cloud_pass"),
        db="bicimad_worker"
    )
    return conn

# create connection pool
pool = sqlalchemy.create_engine(
    "postgresql+pg8000://",
    creator=getconn,
)

In [39]:
def get_disponibilidad(): #Esta función hace append en la base de datos de aquellas columnas que sí cambian a lo largo del día
    connector = Connector()

    # function to return the database connection
    def getconn() -> pg8000.native.Connection:
        conn: pg8000.connections.Connection = connector.connect(
            os.environ.get("google_cloud_project"),
            "pg8000",
            user=os.environ.get("google_cloud_user"),
            password=os.environ.get("google_cloud_pass"),
            db="bicimad_worker"
        )
        return conn

    # create connection pool
    pool = sqlalchemy.create_engine(
        "postgresql+pg8000://",
        creator=getconn,
    )
    
    load_dotenv('./.env')
    token = os.environ.get("access_token")
    url = "https://openapi.emtmadrid.es/v3/transport/bicimad/stations/"
    headers = {"accessToken" : token}
    stations = requests.get(url, headers = headers).json()
    date_and_time = datetime.datetime.now()
    date_and_time_formated = date_and_time.strftime("%Y-%m-%d %H:%M:%S")
    date_and_time_formated2 = date_and_time.strftime("%Y%m%d%H%M%S")
    estaciones = pd.DataFrame(stations["data"])
    estaciones["last_updated"] = date_and_time_formated
    estaciones[["longitude", "latitude"]] = estaciones["geometry"].apply(lambda x: pd.Series(x["coordinates"]))
    estaciones = estaciones.drop(["virtualDelete", "tipo_estacionPBSC", "geofence", "geometry", "integrator", "virtual_bikes", "virtual_bikes_num", "geofenced_capacity", "bikesGo"], axis=1)
    estaciones['coordinates'] = list(zip(estaciones['longitude'], estaciones['latitude']))
    disponibilidad = estaciones[["activate","dock_bikes", "free_bases", "id", "light", "no_available", "reservations_count", "last_updated"]]
    disponibilidad.to_csv(f'../data_csv/disponibilidad_{date_and_time_formated2}.csv')
    with pool.connect() as conn:
        disponibilidad.to_sql('disponibilidad', con=conn, if_exists='append', index=False)
    return disponibilidad

In [61]:
update_estaciones()

PendingRollbackError: Can't reconnect until invalid transaction is rolled back.  Please rollback() fully before proceeding (Background on this error at: https://sqlalche.me/e/20/8s2b)

In [64]:
get_disponibilidad()

,activate,dock_bikes,free_bases,id,light,no_available,reservations_count,last_updated
0,1,10,17,1406,2,0,0,2024-03-13 18:33:38
1,1,0,19,1407,0,0,0,2024-03-13 18:33:38
2,1,17,10,1409,2,0,0,2024-03-13 18:33:38
3,1,7,12,1410,2,0,0,2024-03-13 18:33:38
4,1,5,14,1411,0,0,0,2024-03-13 18:33:38
...,...,...,...,...,...,...,...,...
607,1,4,19,1965,0,0,0,2024-03-13 18:33:38
608,1,6,13,2117,0,0,0,2024-03-13 18:33:38
609,1,19,4,1647,1,0,0,2024-03-13 18:33:38
610,1,19,8,1453,1,0,0,2024-03-13 18:33:38


In [65]:
#Esto hace que se ejecute la función cada hora y se carguen los datos en mi base de datos
schedule.every().hour.do(get_disponibilidad)

while True:
    schedule.run_pending()
    time.sleep(1)

schedule.every().month.do(update_estaciones)

while True:
    schedule.run_pending()
    time.sleep(1)